In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from LSTM_model import LSTMModel
from torch.utils.data import DataLoader, Dataset

In [2]:
df = pd.read_csv("distribution_business.csv")
df.head()

,INV,INVOICE,C/CODE,CUSTOMER NAME,CUSTOMER ADDRESS,ITEM,ITEMS NAME,CTN RATE,QUANTITY,Unnamed: 9,T/O,NET VALUE
0,#,DATE,NaN,NAME,NaN,CODE,NaN,NaN,CTN,PCS,NaN,NaN
1,3,10-Sep-21,9295.0,BIG BITE FAST FOOD,WARSAK ROAD,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500.0,1,0,1000.0,13500.0
2,3,10-Sep-21,9295.0,BIG BITE FAST FOOD,WARSAK ROAD,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250.0,0,5,375.0,9250.0
3,4,10-Sep-21,8992.0,PIZZA FLAME,IBRAHIM MARKET,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500.0,0,2,200.0,2700.0
4,4,10-Sep-21,8992.0,PIZZA FLAME,IBRAHIM MARKET,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250.0,0,2,150.0,3700.0


In [3]:
df = pd.read_csv('distribution_business.csv',header=1,names=["No","Date","C_id","Customer_name","Customer_address","I_code","Product_name","CTN_price","CTN","PCS","Total_discount","Net_value"],parse_dates=["Date"])
df.head()

/tmp/ipykernel_38979/2161219627.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('distribution_business.csv',header=1,names=["No","Date","C_id","Customer_name","Customer_address","I_code","Product_name","CTN_price","CTN","PCS","Total_discount","Net_value"],parse_dates=["Date"])


,No,Date,C_id,Customer_name,Customer_address,I_code,Product_name,CTN_price,CTN,PCS,Total_discount,Net_value
0,3,2021-09-10,9295,BIG BITE FAST FOOD,WARSAK ROAD,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,1,0,1000,13500.0
1,3,2021-09-10,9295,BIG BITE FAST FOOD,WARSAK ROAD,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250,0,5,375,9250.0
2,4,2021-09-10,8992,PIZZA FLAME,IBRAHIM MARKET,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,0,2,200,2700.0
3,4,2021-09-10,8992,PIZZA FLAME,IBRAHIM MARKET,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250,0,2,150,3700.0
4,5,2021-09-10,9903,CHIL & GRILL 0312.9553655,MOHMAD ZAI,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,0,2,200,2700.0


In [4]:
df = df.drop("No", axis=1)

In [5]:
df = df.drop(df['Net_value'].nsmallest(11).index)

min_rows = df.nsmallest(5, ['Net_value'])

#  the resulting dataframe
(min_rows)

,Date,C_id,Customer_name,Customer_address,I_code,Product_name,CTN_price,CTN,PCS,Total_discount,Net_value
92,2021-09-14,2132,FREE SAMPLING,NaN,1763,DEEN'S SLICE CHEESE 200GM,23600,0,1,0,295.0
192,2021-09-21,9556,EASY MART,ZANGAL MARKET,1763,DEEN'S SLICE CHEESE 200GM,23600,0,1,0,295.0
216,2021-09-21,8898,SS SUPER STORE,COL SHER KHAN MARKET,1763,DEEN'S SLICE CHEESE 200GM,23600,0,1,0,295.0
291,2021-09-25,2025,GUL G/S,Behram Market,1763,DEEN'S SLICE CHEESE 200GM,23600,0,1,0,295.0
429,2021-10-02,9031,UNIVERSITY BAKERS 03335992712,PESHAWAR UNIVERSITY,1763,DEEN'S SLICE CHEESE 200GM,23600,0,1,0,295.0


In [6]:
df = df.drop(df['Total_discount'].nsmallest(2).index)

min_rows = df.nsmallest(5, ['Total_discount'])
#df = df.drop(min_rows.index)

#  the resulting dataframe
(min_rows)

,Date,C_id,Customer_name,Customer_address,I_code,Product_name,CTN_price,CTN,PCS,Total_discount,Net_value
6,2021-09-11,6544,CHATRAL,CAMPUS,1758,DEEN'S CHEDDAR 200GM,20150,0,4,0,1240.0
7,2021-09-11,6544,CHATRAL,CAMPUS,1760,DEEN'S MOZARELLA 200GM,22425,0,6,0,2070.0
8,2021-09-11,6544,CHATRAL,CAMPUS,1762,DEEN'S PIZZA CHEESE 200GM,24050,0,2,0,740.0
9,2021-09-11,6544,CHATRAL,CAMPUS,1763,DEEN'S SLICE CHEESE 200GM,23600,0,2,0,590.0
10,2021-09-11,3467,IQRA SWEET & BAKERS,CAMPUS,1758,DEEN'S CHEDDAR 200GM,20150,0,1,0,310.0


In [7]:
(df.nunique())

Date                373
C_id                221
Customer_name       214
Customer_address    131
I_code               14
Product_name         16
CTN_price            43
CTN                  11
PCS                  24
Total_discount      336
Net_value           731
dtype: int64

In [8]:
('Number of rows:', df.shape[0])
('Number of columns:', df.shape[1])

('Number of columns:', 11)

In [9]:
df.describe()

,Date,C_id,I_code,CTN_price,CTN,PCS,Total_discount,Net_value
count,7282,7282.000000,7282.000000,7282.000000,7282.000000,7282.000000,7282.000000,7282.000000
mean,2022-04-06 11:03:14.781653248,5189.002060,1764.258857,23851.971024,0.142131,3.148860,185.013321,5010.439247
min,2021-09-10 00:00:00,2005.000000,1758.000000,14400.000000,0.000000,0.000000,0.000000,295.000000
25%,2021-12-23 00:00:00,2136.000000,1760.000000,21180.000000,0.000000,2.000000,0.000000,902.000000
50%,2022-03-30 00:00:00,3403.000000,1763.000000,23725.000000,0.000000,2.000000,0.000000,1500.000000
75%,2022-07-18 00:00:00,9024.000000,1770.000000,26650.000000,0.000000,4.000000,90.000000,2904.000000
max,2022-12-03 00:00:00,9965.000000,1788.000000,36006.000000,12.000000,65.000000,24220.000000,258600.000000
std,NaN,3348.983995,5.410203,4821.487524,0.875108,3.310566,927.670823,17730.800894


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7282 entries, 0 to 7294
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              7282 non-null   datetime64[ns]
 1   C_id              7282 non-null   int64         
 2   Customer_name     7282 non-null   object        
 3   Customer_address  7033 non-null   object        
 4   I_code            7282 non-null   int64         
 5   Product_name      7282 non-null   object        
 6   CTN_price         7282 non-null   int64         
 7   CTN               7282 non-null   int64         
 8   PCS               7282 non-null   int64         
 9   Total_discount    7282 non-null   int64         
 10  Net_value         7282 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(6), object(3)
memory usage: 682.7+ KB


In [11]:
df.isnull().sum()

Date                  0
C_id                  0
Customer_name         0
Customer_address    249
I_code                0
Product_name          0
CTN_price             0
CTN                   0
PCS                   0
Total_discount        0
Net_value             0
dtype: int64

In [12]:
df.dropna(subset=['Customer_address'], inplace=True)

In [13]:
df.isnull().sum()

Date                0
C_id                0
Customer_name       0
Customer_address    0
I_code              0
Product_name        0
CTN_price           0
CTN                 0
PCS                 0
Total_discount      0
Net_value           0
dtype: int64

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
df.duplicated().sum()

0

In [16]:
(df.dtypes)

Date                datetime64[ns]
C_id                         int64
Customer_name               object
Customer_address            object
I_code                       int64
Product_name                object
CTN_price                    int64
CTN                          int64
PCS                          int64
Total_discount               int64
Net_value                  float64
dtype: object

In [17]:
(df["Product_name"].unique())

array(["DEEN'S CHADDER CHEESE BLOCK 2K", "DEEN'S MOZRELLA CHEESE BLOCK",
       "DEEN'S CHEDDAR 200GM", "DEEN'S MOZARELLA 200GM",
       "DEEN'S PIZZA CHEESE 200GM", "DEEN'S SLICE CHEESE 200GM",
       "DEEN'S CHEDDAR 400GM", "DEEN'S MOZARELLA 400GM",
       "DEEN'S SLICE CHEESE 1 KG", 'w4-Dyh SLICE CHEESE 1 KG',
       'DEEN SLICE CHEESE 1 KG', 'DEENS CHADDER 140G',
       'DEENS CHADDER 907G', 'DEENS MOZRELLA 140G', 'DEENS PIZZA 400G',
       'DEENS SLICE 250G'], dtype=object)

In [18]:
(df["CTN"].value_counts())

CTN
0     6737
1      102
5       43
2       41
4       27
3       19
10      18
6       16
7       12
8        7
12       1
Name: count, dtype: int64

In [19]:
count_10 = (df['PCS'] == 10).sum()
(f"There are {count_10} rows with a value of 10 in the 'PCS' column.")

"There are 33 rows with a value of 10 in the 'PCS' column."

In [20]:
product_quantities = {
    "DEEN'S CHADDER CHEESE BLOCK 2K": 10,
    "DEEN'S MOZRELLA CHEESE BLOCK": 10,
    "DEEN'S CHEDDAR 200GM": 72,
    "DEEN'S MOZARELLA 200GM": 72,
    "DEEN'S PIZZA CHEESE 200GM": 72,
    "DEEN'S SLICE CHEESE 200GM": 80,
    "DEEN'S CHEDDAR 400GM": 36,
    "DEEN'S MOZARELLA 400GM": 36,
    "DEEN'S SLICE CHEESE 1 KG": 16,
    "w4-Dyh SLICE CHEESE 1 KG": 16,
    "DEEN SLICE CHEESE 1 KG": 16,
    "DEENS CHADDER 140G": 102,
    "DEENS CHADDER 907G": 18,
    "DEENS MOZRELLA 140G": 102,
    "DEENS PIZZA 400G": 36,
    "DEENS SLICE 250G": 66
}
   
for ctn in range(1, 9):
    for product, quantity in product_quantities.items():
        df.loc[(df['Product_name'] == product) & (df['CTN'] == ctn), 'PCS'] = quantity * ctn
    
for ctn in range(10, 13,2):
    for product, quantity in product_quantities.items():
        df.loc[(df['Product_name'] == product) & (df['CTN'] == ctn), 'PCS'] = quantity * ctn

In [21]:
def get_pcs_in_ctn(product_name):
    if "DEEN'S CHADDER CHEESE BLOCK 2K" in product_name:
        return 10  
    elif "DEEN'S MOZRELLA CHEESE BLOCK" in product_name:
        return 10
    elif "DEEN'S CHEDDAR 200GM" in product_name:
        return 72 
    elif "DEEN'S MOZARELLA 200GM" in product_name:
        return 72
    elif "DEEN'S PIZZA CHEESE 200GM" in product_name:
        return 72
    elif "DEEN'S SLICE CHEESE 200GM" in product_name:
        return 80
    elif "400G" in product_name:
        return 36
    elif "1 KG" in product_name:
        return 16
    elif "140G" in product_name:
        return 102
    elif "DEENS CHADDER 907G" in product_name:
        return 18
    elif "DEENS SLICE 250G" in product_name:
        return 66
    else:
        return 0  # Return 0 if the product name doesn't match any known pattern

# Add a new column 'PCS_in_CTN' based on the 'Product_name' column
df['PCS_in_CTN'] = df['Product_name'].apply(get_pcs_in_ctn)

Create a new column called "Price_per_CTN" by dividing "Orignal_price" by "PCS_in_CTN"

In [22]:
df['Price_per_PCS'] = df['CTN_price'] / df['PCS_in_CTN']

Creating a new feature 'Discount_per_piece'

In [23]:
df['Discount_per_piece'] = df['Total_discount'] / df['PCS']

Creating a new feature 'Month'

In [24]:
df['Month'] = pd.DatetimeIndex(df['Date']).month

In [25]:
df['Year'] = pd.DatetimeIndex(df['Date']).year

Creating a new feature 'Has_Discount'

In [26]:
df["Has_discount"] = (df["Total_discount"] > 0).astype(int)

In [27]:
df["Product_name"] = df["Product_name"].replace('DEEN SLICE CHEESE 1 KG', "DEEN'S SLICE CHEESE 1 KG")

### Define the constant purchase price values for the specified products

start_date = "2021-09-10"


end_date = "2021-12-22"

In [28]:
product_prices = {
    "DEEN'S CHADDER CHEESE BLOCK 2K": 12282,
    "DEEN'S MOZRELLA CHEESE BLOCK": 16362,
    "DEEN'S CHEDDAR 200GM": 17127,
    "DEEN'S MOZARELLA 200GM": 19061,
    "DEEN'S PIZZA CHEESE 200GM": 20442,
    "DEEN'S SLICE CHEESE 200GM": 20060,
    "DEEN'S CHEDDAR 400GM": 16770,
    "DEEN'S MOZARELLA 400GM": 18326,
    "DEEN'S SLICE CHEESE 1 KG": 12240,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}
start_date = "2021-09-10"
end_date = "2021-12-22"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = None  # Create the "Purchase_price" column

df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2021-12-22"

end_date = "2022-01-25"

In [29]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHADDER CHEESE BLOCK 2K": 12282,
    "DEEN'S MOZRELLA CHEESE BLOCK": 16362,
    "DEEN'S CHEDDAR 200GM": 17127,
    "DEEN'S MOZARELLA 200GM": 19061,
    "DEEN'S PIZZA CHEESE 200GM": 20442,
    "DEEN'S SLICE CHEESE 200GM": 20060,
    "DEEN'S CHEDDAR 400GM": 16770,
    "DEEN'S MOZARELLA 400GM": 18326,
    "DEEN'S SLICE CHEESE 1 KG": 12240,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2021-12-22"
end_date = "2022-01-25"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-01-25"

end_date = "2022-01-31"

In [30]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 18785,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 20939,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 22486,
    "DEEN'S SLICE CHEESE 200GM": 22032,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 13515,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18003,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-01-25"
end_date = "2022-01-31"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-01-31"

end_date = "2022-03-07"

In [31]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 18785,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 20939,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 22486,
    "DEEN'S SLICE CHEESE 200GM": 22032,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 13515,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18003,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-01-31"
end_date = "2022-03-07"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-03-07"

end_date = "2022-04-14"


In [32]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 18785,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 20939,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 22486,
    "DEEN'S SLICE CHEESE 200GM": 22032,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 13515,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18003,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-03-07"
end_date = "2022-04-14"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-04-14"

end_date = "2022-05-30"


In [33]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 20166,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 22652,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 24033,
    "DEEN'S SLICE CHEESE 200GM": 23800,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 13515,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18003,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-04-14"
end_date = "2022-05-30"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-05-30"

end_date = "2022-06-04

In [34]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 20166,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 22652,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 24033,
    "DEEN'S SLICE CHEESE 200GM": 23800,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 13515,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18003,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-05-30"
end_date = "2022-06-04"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-06-04"

end_date = "2022-06-13"

In [35]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 20166,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 22652,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 24033,
    "DEEN'S SLICE CHEESE 200GM": 23800,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 14365,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18853,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-06-04"
end_date = "2022-06-13"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-06-13"

end_date = "2022-06-16"

In [36]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 20166,
    "DEEN'S CHEDDAR 400GM": 19339,
    "DEEN'S MOZARELLA 200GM": 22652,
    "DEEN'S MOZARELLA 400GM": 21879,
    "DEEN'S PIZZA CHEESE 200GM": 24033,
    "DEEN'S SLICE CHEESE 200GM": 23800,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 14365,
    "DEEN'S MOZRELLA CHEESE BLOCK": 18853,
    "DEEN'S SLICE CHEESE 1 KG": 13464,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-06-13"
end_date = "2022-06-16"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-06-16"

end_date = "2022-07-06"

In [37]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 22431,
    "DEEN'S CHEDDAR 400GM": 23011,
    "DEEN'S MOZARELLA 200GM": 24917,
    "DEEN'S MOZARELLA 400GM": 26010,
    "DEEN'S PIZZA CHEESE 200GM": 26741,
    "DEEN'S SLICE CHEESE 200GM": 26248,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 20300,
    "DEEN'S MOZRELLA CHEESE BLOCK": 23270,
    "DEEN'S SLICE CHEESE 1 KG": 15840,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-06-16"
end_date = "2022-07-06"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-07-06"

end_date = "2022-07-14"

In [38]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 21845,
    "DEEN'S CHEDDAR 400GM": 22440,
    "DEEN'S MOZARELLA 200GM": 24312,
    "DEEN'S MOZARELLA 400GM": 25317,
    "DEEN'S PIZZA CHEESE 200GM": 26072,
    "DEEN'S SLICE CHEESE 200GM": 24487,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 20300,
    "DEEN'S MOZRELLA CHEESE BLOCK": 23270,
    "DEEN'S SLICE CHEESE 1 KG": 15840,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-07-06"
end_date = "2022-07-14"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-07-14"

end_date = "2022-11-04"

In [39]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 21845,
    "DEEN'S CHEDDAR 400GM": 22440,
    "DEEN'S MOZARELLA 200GM": 24312,
    "DEEN'S MOZARELLA 400GM": 25317,
    "DEEN'S PIZZA CHEESE 200GM": 26072,
    "DEEN'S SLICE CHEESE 200GM": 24487,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 20300,
    "DEEN'S MOZRELLA CHEESE BLOCK": 23270,
    "DEEN'S SLICE CHEESE 1 KG": 15840,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
}

# Define the start and end dates for the constant price period
start_date = "2022-07-14"
end_date = "2022-11-04"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


start_date = "2022-11-04"

end_date = "2022-12-04"

In [40]:
# Define the constant purchase price values for the specified products
product_prices = {
    "DEEN'S CHEDDAR 200GM": 24840,
    "DEEN'S CHEDDAR 400GM": 22440,
    "DEEN'S MOZARELLA 200GM": 24895,
    "DEEN'S MOZARELLA 400GM": 25371,
    "DEEN'S PIZZA CHEESE 200GM": 29592,
    "DEEN'S SLICE CHEESE 200GM": 26240,
    "DEEN'S CHADDER CHEESE BLOCK 2K": 20128,
    "DEEN'S MOZRELLA CHEESE BLOCK": 23069,
    "DEEN'S SLICE CHEESE 1 KG": 15840,
    'w4-Dyh SLICE CHEESE 1 KG':0,
    'DEENS CHADDER 140G':0,
    'DEENS CHADDER 907G':0,
    'DEENS MOZRELLA 140G':0,
    'DEENS PIZZA 400G':0,
    'DEENS SLICE 250G':0
    
}

# Define the start and end dates for the constant price period
start_date = "2022-11-04"
end_date = "2022-12-04"

# Create the Purchase_price column and set the constant prices for the specified products within the date range
df["Purchase_price"] = df.apply(lambda row: product_prices[row["Product_name"]] if (row["Product_name"] in product_prices.keys() and start_date <= str(row["Date"]) <= end_date) else row["Purchase_price"], axis=1)


In [41]:
df["Profit_per_order"] = df["Net_value"] - df["Purchase_price"]
df["Profit_per_order"] = (df["Purchase_price"] / df["PCS_in_CTN"]) * df["PCS"]
df["Profit_per_order"] = df["Net_value"] - df["Profit_per_order"]

In [42]:
df["Product_name"].value_counts()

Product_name
DEEN'S MOZARELLA 200GM            1332
DEEN'S SLICE CHEESE 200GM         1208
DEEN'S CHEDDAR 200GM              1128
DEEN'S CHADDER CHEESE BLOCK 2K    1114
DEEN'S MOZRELLA CHEESE BLOCK       971
DEEN'S PIZZA CHEESE 200GM          818
DEEN'S SLICE CHEESE 1 KG           274
DEENS SLICE 250G                    57
DEEN'S MOZARELLA 400GM              55
DEEN'S CHEDDAR 400GM                44
DEENS MOZRELLA 140G                 10
DEENS CHADDER 140G                   7
DEENS CHADDER 907G                   2
DEENS PIZZA 400G                     2
w4-Dyh SLICE CHEESE 1 KG             1
Name: count, dtype: int64

In [43]:
df["Price_per_PCS"] = df["Price_per_PCS"].astype("int64")
df["Discount_per_piece"] = df["Discount_per_piece"].astype("int64")
df["Has_discount"] = df["Has_discount"].astype("int64")
df["Net_value"] = df["Net_value"].astype("int64")
df["Purchase_price"] = df["Purchase_price"].astype("int64")
df["Profit_per_order"] = df["Profit_per_order"].astype("int64")

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7023 entries, 0 to 7294
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                7023 non-null   datetime64[ns]
 1   C_id                7023 non-null   int64         
 2   Customer_name       7023 non-null   object        
 3   Customer_address    7023 non-null   object        
 4   I_code              7023 non-null   int64         
 5   Product_name        7023 non-null   object        
 6   CTN_price           7023 non-null   int64         
 7   CTN                 7023 non-null   int64         
 8   PCS                 7023 non-null   int64         
 9   Total_discount      7023 non-null   int64         
 10  Net_value           7023 non-null   int64         
 11  PCS_in_CTN          7023 non-null   int64         
 12  Price_per_PCS       7023 non-null   int64         
 13  Discount_per_piece  7023 non-null   int64         
 1

### Get insights from data

Identify the top-selling products

In [45]:
top_products = df.groupby(['I_code', 'Product_name'])['PCS'].sum().reset_index().sort_values('PCS', ascending=False).head(10)
top_products

,I_code,Product_name,PCS
6,1770,DEEN'S CHADDER CHEESE BLOCK 2K,9020
2,1760,DEEN'S MOZARELLA 200GM,5424
5,1763,DEEN'S SLICE CHEESE 200GM,5000
7,1771,DEEN'S MOZRELLA CHEESE BLOCK,4830
0,1758,DEEN'S CHEDDAR 200GM,4073
4,1762,DEEN'S PIZZA CHEESE 200GM,2693
8,1772,DEEN'S SLICE CHEESE 1 KG,684
10,1784,DEENS SLICE 250G,337
1,1759,DEEN'S CHEDDAR 400GM,312
3,1761,DEEN'S MOZARELLA 400GM,246


#Analyze the discounts

In [46]:
discount_analysis = df.groupby('Product_name').agg({'Total_discount': 'mean', 'Net_value': 'mean', 'PCS': 'sum'}).reset_index()
discount_analysis

,Product_name,Total_discount,Net_value,PCS
0,DEEN'S CHADDER CHEESE BLOCK 2K,608.484740,15087.789946,9020
1,DEEN'S CHEDDAR 200GM,17.747340,1288.093972,4073
2,DEEN'S CHEDDAR 400GM,464.909091,4375.272727,312
3,DEEN'S MOZARELLA 200GM,28.720721,1616.367117,5424
4,DEEN'S MOZARELLA 400GM,100.872727,3260.218182,246
5,DEEN'S MOZRELLA CHEESE BLOCK,465.107106,10866.084449,4830
6,DEEN'S PIZZA CHEESE 200GM,20.908313,1393.641809,2693
7,DEEN'S SLICE CHEESE 1 KG,42.518248,2441.021898,684
8,DEEN'S SLICE CHEESE 200GM,19.701987,1395.638245,5000
9,DEENS CHADDER 140G,0.000000,5087.428571,116


Identify the most profitable customers

In [47]:
top_customers = df.groupby(['Customer_name', 'Customer_address'])[['Net_value', 'PCS']].sum().reset_index().sort_values('PCS', ascending=False).head(10)
top_customers

,Customer_name,Customer_address,Net_value,PCS
41,CHIEF BURGER,UNIVERSITY ROAD,13830755,6763
95,JAN'S DAILY,TOWN,2953450,1550
145,PIZZA FLAME,IBRAHIM MARKET,2152055,1254
85,IMTIAZ SUPER STORE,RING ROAD,693975,1097
9,AL HAMRA SUPER STORE,DILZAQ ROAD,320531,887
18,ARYANA S/S,Nawab Market,312446,811
70,H.B.K Hypermarket (Pvt.) Ltd.,ring road,376621,744
177,SNACK FOOD CAFE,ZANGAL MARKET,1034831,598
185,TOP TOWN,TORANGZAI MARKET HAYATBAD,192666,531
55,FAISAL STORE,Yousaf Zai Market,190583,510


Analyze the pricing strategy

In [48]:
price_analysis = df.groupby('Product_name').agg({'CTN_price': 'mean', 'Net_value': 'mean', 'PCS': 'sum'}).reset_index()
price_analysis

,Product_name,CTN_price,Net_value,PCS
0,DEEN'S CHADDER CHEESE BLOCK 2K,17523.339318,15087.789946,9020
1,DEEN'S CHEDDAR 200GM,23530.681738,1288.093972,4073
2,DEEN'S CHEDDAR 400GM,23134.090909,4375.272727,312
3,DEEN'S MOZARELLA 200GM,26286.412913,1616.367117,5424
4,DEEN'S MOZARELLA 400GM,26158.909091,3260.218182,246
5,DEEN'S MOZRELLA CHEESE BLOCK,22023.233780,10866.084449,4830
6,DEEN'S PIZZA CHEESE 200GM,28055.408313,1393.641809,2693
7,DEEN'S SLICE CHEESE 1 KG,15872.116788,2441.021898,684
8,DEEN'S SLICE CHEESE 200GM,27251.192053,1395.638245,5000
9,DEENS CHADDER 140G,31314.000000,5087.428571,116


Analyze customer behavior

In [49]:
customer_behavior = df.groupby(['Customer_name', 'Customer_address']).agg({'CTN': 'sum', 'PCS': 'sum'}).reset_index()
customer_behavior['Items_per_carton'] = customer_behavior['PCS'] / customer_behavior['CTN']
customer_behavior

,Customer_name,Customer_address,CTN,PCS,Items_per_carton
0,4 SEASON,SADAR,0,251,inf
1,7 SEAZ S S,KHYBER SUPER MARKET,0,23,inf
2,AD STORE,Colonel Sher Khan Market,0,90,inf
3,ADNAN G/S,M ZAI MARKET,0,48,inf
4,ADNAN GENERAL STORE,ARY BAZAR,0,2,inf
...,...,...,...,...,...
194,WARSAK MALL,WARSAK ROAD,0,36,inf
195,YOUNAS GENERAL STORE,GUL BERG 2,2,398,199.0
196,ZAFAR G/S,HAYATABAD TURANGZAI MARKET,0,256,inf
197,ZAHID MOHMAND,NOTHIA,0,1,inf


Identify correlations

In [50]:
corr_matrix = df[['CTN_price', 'CTN', 'PCS', 'Total_discount', 'Net_value']].corr()
corr_matrix

,CTN_price,CTN,PCS,Total_discount,Net_value
CTN_price,1.000000,-0.096463,-0.008036,-0.161988,-0.101409
CTN,-0.096463,1.000000,0.867981,0.747191,0.977248
PCS,-0.008036,0.867981,1.000000,0.678617,0.877153
Total_discount,-0.161988,0.747191,0.678617,1.000000,0.683084
Net_value,-0.101409,0.977248,0.877153,0.683084,1.000000


## Modeling 

## Feature Engineering

#### Creating Day column

Droping Columns/features

In [51]:
df.columns

Index(['Date', 'C_id', 'Customer_name', 'Customer_address', 'I_code',
       'Product_name', 'CTN_price', 'CTN', 'PCS', 'Total_discount',
       'Net_value', 'PCS_in_CTN', 'Price_per_PCS', 'Discount_per_piece',
       'Month', 'Year', 'Has_discount', 'Purchase_price', 'Profit_per_order'],
      dtype='object')

In [52]:
drops = [
    'C_id',
    'Customer_name',
    'Customer_address',
    'Customer_address', 
]

In [53]:
df = df.drop(drops, axis=1)

 Extract the day from the 'Date' column and assign it to the 'Day' column

In [54]:
df['Day'] = df['Date'].dt.day

Creating new column Month_order (Total no. of orders in a month)

In [55]:
product_names = [
    "DEEN'S CHADDER CHEESE BLOCK 2K",
    "DEEN'S MOZRELLA CHEESE BLOCK",
    "DEEN'S CHEDDAR 200GM",
    "DEEN'S MOZARELLA 200GM",
    "DEEN'S PIZZA CHEESE 200GM",
    "DEEN'S SLICE CHEESE 200GM",
    "DEEN'S CHEDDAR 400GM",
    "DEEN'S MOZARELLA 400GM",
    "DEEN'S SLICE CHEESE 1 KG"
]

date_ranges = [
    ('2021-09-01', '2021-09-30'),
    ('2021-10-01', '2021-10-31'),
    ('2021-11-01', '2021-11-30'),
    ('2021-12-01', '2021-12-31'),
    ('2022-01-01', '2022-01-31'),
    ('2022-02-01', '2022-02-28'),
    ('2022-03-01', '2022-03-31'),
    ('2022-04-01', '2022-04-30'),
    ('2022-05-01', '2022-05-31'),
    ('2022-06-01', '2022-06-30'),
    ('2022-07-01', '2022-07-31'),
    ('2022-08-01', '2022-08-31'),
    ('2022-09-01', '2022-09-30'),
    ('2022-10-01', '2022-10-31'),
    ('2022-11-01', '2022-12-03')
]

# Convert the 'Date' column to datetime type if not already done
df['Date'] = pd.to_datetime(df['Date'])

# Create a new column 'month_order' and initialize it with zeros
df['month_order'] = 0

# Iterate over each product name and date range
for product_name in product_names:
    for start_date, end_date in date_ranges:
        # Create a mask for the specified date range
        date_mask = (df['Date'].between(start_date, end_date))
        
        # Filter the DataFrame for the specified product and date range
        filtered_df = df[(df['Product_name'] == product_name) & date_mask]
        
        # Calculate the total number of PCS ordered
        total_pcs_ordered = filtered_df['PCS'].sum()
        
        # Update the 'month_order' column for the specific product and date range
        df.loc[(df['Product_name'] == product_name) & date_mask, 'month_order'] = total_pcs_ordered

Creating new Column week_order (Total no. of pieces ordered in a week)

### Convert the 'Date' column to datetime type if not already done

In [56]:
df['Date'] = pd.to_datetime(df['Date'])

Create a new column 'Week' and extract the week number within each month

In [57]:
df['Week'] = df['Date'].dt.isocalendar().week

 Iterate over each month and update the week numbers accordingly

In [58]:
for month in df['Month'].unique():
    mask = (df['Month'] == month)
    df.loc[mask, 'Week'] = df.loc[mask, 'Week'].rank(method='dense')

Define the list of specific product names

In [59]:

product_names = ["DEEN'S CHADDER CHEESE BLOCK 2K", "DEEN'S MOZRELLA CHEESE BLOCK",
                 "DEEN'S CHEDDAR 200GM", "DEEN'S MOZARELLA 200GM",
                 "DEEN'S PIZZA CHEESE 200GM", "DEEN'S SLICE CHEESE 200GM",
                 "DEEN'S CHEDDAR 400GM", "DEEN'S MOZARELLA 400GM",
                 "DEEN'S SLICE CHEESE 1 KG"]

Create a new column 'week_order' and initialize it with 0

In [60]:
df['week_order'] = 0

Iterate over each product name:

Filter the dataframe for the specific product

Group the filtered data by week and calculate the sum of PCS ordered

Update the 'week_order' column with the sum of PCS within each week for the specific product

In [61]:
for product_name in product_names:
    filtered_df = df[df['Product_name'] == product_name]
    weekly_pcs_ordered = filtered_df.groupby(['Year', 'Month', 'Week'])['PCS'].sum()
    df.loc[df['Product_name'] == product_name, 'week_order'] = df.loc[df['Product_name'] == product_name].groupby(['Year', 'Month', 'Week'])['PCS'].transform('sum')

 Sort the dataframe by date

In [62]:
df = df.sort_values('Date')

Reset the index

In [63]:
df = df.reset_index(drop=True)

### Creating Season cloumn

Define a function to map the month to the corresponding season

In [64]:
# Define a function to map the month to the corresponding season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4]:
        return 'Spring'
    elif month in [5, 6, 7, 8, 9]:
        return 'Summer'
    else:
        return 'Autumn'

# Apply the function to the 'Month' column and assign the result to the 'Season' column
df['Season'] = df['Month'].apply(get_season)


In [65]:
# Create a dictionary to map seasons to numerical values
season_mapping = {'Winter': 1, 'Spring': 2, 'Summer': 3, 'Autumn': 4}

# Map the seasons to numerical values and assign the result to a new column 'Season_encoded'
df['Season_encoded'] = df['Season'].map(season_mapping)

In [66]:
# Assuming 'df' is your dataset dataframe
column_with_value = None

for column in df.columns:
    if 'BIG BITE FAST FOOD' in df[column].values:
        column_with_value = column
        break

# Print the column with the desired value
print("Column with 'BIG BITE FAST FOOD':", column_with_value)


Column with 'BIG BITE FAST FOOD': None


In [67]:
df.head()

,Date,I_code,Product_name,CTN_price,CTN,PCS,Total_discount,Net_value,PCS_in_CTN,Price_per_PCS,...,Year,Has_discount,Purchase_price,Profit_per_order,Day,month_order,Week,week_order,Season,Season_encoded
0,2021-09-10,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,1,10,1000,13500,10,1450,...,2021,1,12282,1218,10,142,2,18,Summer,3
1,2021-09-10,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250,0,5,375,9250,10,1925,...,2021,1,16362,1069,10,103,2,13,Summer,3
2,2021-09-10,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,0,2,200,2700,10,1450,...,2021,1,12282,243,10,142,2,18,Summer,3
3,2021-09-10,1771,DEEN'S MOZRELLA CHEESE BLOCK,19250,0,2,150,3700,10,1925,...,2021,1,16362,427,10,103,2,13,Summer,3
4,2021-09-10,1770,DEEN'S CHADDER CHEESE BLOCK 2K,14500,0,2,200,2700,10,1450,...,2021,1,12282,243,10,142,2,18,Summer,3


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                7023 non-null   datetime64[ns]
 1   I_code              7023 non-null   int64         
 2   Product_name        7023 non-null   object        
 3   CTN_price           7023 non-null   int64         
 4   CTN                 7023 non-null   int64         
 5   PCS                 7023 non-null   int64         
 6   Total_discount      7023 non-null   int64         
 7   Net_value           7023 non-null   int64         
 8   PCS_in_CTN          7023 non-null   int64         
 9   Price_per_PCS       7023 non-null   int64         
 10  Discount_per_piece  7023 non-null   int64         
 11  Month               7023 non-null   int32         
 12  Year                7023 non-null   int32         
 13  Has_discount        7023 non-null   int64       

In [69]:
df['Week'] = df['Week'].astype(np.int32)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                7023 non-null   datetime64[ns]
 1   I_code              7023 non-null   int64         
 2   Product_name        7023 non-null   object        
 3   CTN_price           7023 non-null   int64         
 4   CTN                 7023 non-null   int64         
 5   PCS                 7023 non-null   int64         
 6   Total_discount      7023 non-null   int64         
 7   Net_value           7023 non-null   int64         
 8   PCS_in_CTN          7023 non-null   int64         
 9   Price_per_PCS       7023 non-null   int64         
 10  Discount_per_piece  7023 non-null   int64         
 11  Month               7023 non-null   int32         
 12  Year                7023 non-null   int32         
 13  Has_discount        7023 non-null   int64       

Define the dataset class

In [71]:
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = torch.tensor(self.data[index], dtype=torch.float32)
        y = torch.tensor(self.targets[index], dtype=torch.float32)
        return x, y

Define the inputs and outputs

In [72]:
inputs = df[['I_code', 'Day', 'Month', 'Week', 'Year', 'PCS', 'Season_encoded', 'Has_discount']].values
outputs = df[['month_order', 'Net_value', 'week_order', 'Profit_per_order']].values

Normalize the inputs and outputs

In [73]:
inputs = (inputs - inputs.min(axis=0)) / (inputs.max(axis=0) - inputs.min(axis=0))
outputs = (outputs - outputs.min(axis=0)) / (outputs.max(axis=0) - outputs.min(axis=0))

Split the data into training and testing sets

In [74]:
train_size = int(len(inputs) * 0.8)
train_inputs = inputs[:train_size]
train_outputs = outputs[:train_size]
test_inputs = inputs[train_size:]
test_outputs = outputs[train_size:]

Create DataLoader for training and testing sets

In [75]:
train_dataset = CustomDataset(train_inputs, train_outputs)
test_dataset = CustomDataset(test_inputs, test_outputs)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Set the device

In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Initialize the model and move it to the device

In [77]:
input_size = inputs.shape[1]
output_size = outputs.shape[1]
hidden_size = 64
model = LSTMModel(input_size, hidden_size, output_size).to(device)

Define the loss function and optimizer

In [78]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train the model

In [79]:
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}')

/home/usamah/anaconda3/envs/distribution_businesss/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/usamah/anaconda3/envs/distribution_businesss/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([18, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 0.0244
Epoch 2/50, Loss: 0.0203
Epoch 3/50, Loss: 0.0203
Epoch 4/50, Loss: 0.0202
Epoch 5/50, Loss: 0.0202
Epoch 6/50, Loss: 0.0201
Epoch 7/50, Loss: 0.0202
Epoch 8/50, Loss: 0.0200
Epoch 9/50, Loss: 0.0202
Epoch 10/50, Loss: 0.0201
Epoch 11/50, Loss: 0.0201
Epoch 12/50, Loss: 0.0201
Epoch 13/50, Loss: 0.0200
Epoch 14/50, Loss: 0.0200
Epoch 15/50, Loss: 0.0200
Epoch 16/50, Loss: 0.0201
Epoch 17/50, Loss: 0.0201
Epoch 18/50, Loss: 0.0200
Epoch 19/50, Loss: 0.0201
Epoch 20/50, Loss: 0.0200
Epoch 21/50, Loss: 0.0201
Epoch 22/50, Loss: 0.0200
Epoch 23/50, Loss: 0.0200
Epoch 24/50, Loss: 0.0200
Epoch 25/50, Loss: 0.0200
Epoch 26/50, Loss: 0.0201
Epoch 27/50, Loss: 0.0200
Epoch 28/50, Loss: 0.0201
Epoch 29/50, Loss: 0.0200
Epoch 30/50, Loss: 0.0200
Epoch 31/50, Loss: 0.0201
Epoch 32/50, Loss: 0.0200
Epoch 33/50, Loss: 0.0200
Epoch 34/50, Loss: 0.0200
Epoch 35/50, Loss: 0.0201
Epoch 36/50, Loss: 0.0200
Epoch 37/50, Loss: 0.0201
Epoch 38/50, Loss: 0.0200
Epoch 39/50, Loss: 0.

In [80]:
# Make predictions
model.eval()
with torch.no_grad():
    test_inputs = torch.tensor(test_inputs, dtype=torch.float32).unsqueeze(0).to(device)
    predictions = model(test_inputs)

# Denormalize the predictions
predictions = (
    predictions.cpu().detach().numpy()
    * (
        torch.max(outputs, dim=0).values.cpu().detach().numpy()
        - torch.min(outputs, dim=0).values.cpu().detach().numpy()
    )
    + torch.min(outputs, dim=0).values.cpu().detach().numpy()
)

# Print the predicted values
for i, prediction in enumerate(predictions):
    print(f'Prediction {i+1}:')
    print('Monthly Order:', prediction[0])
    print('Net Value:', prediction[1])
    print('Weekly Order:', prediction[2])
    print('Profit per Order:', prediction[3])


Prediction 1:
Monthly Order: 0.12542352
Net Value: 0.022349473
Weekly Order: 0.08038871
Profit per Order: 0.060163826


In [81]:
torch.save(model.state_dict(), 'lstm_model.pth')
print("Model saved successfully.")

Model saved successfully.
